In [2]:
import pandas as pd
from pathlib import Path
import plotly.express as px
import os
import geopandas as gpd
from shapely.geometry import LineString

from neo4j import GraphDatabase

PATH = Path.cwd().parent.joinpath('data')

## Goal

The goal of this notebook is to perform exploratory data analysis on the shadowfleet data and create a model for a graph database. 

### Data description

First we need to know if we have all necessary data and if not, what data we are missing. Our starting point is the shadowfleet data provided by the Kiev School of Economics Institute.

#### KSE

In [3]:
kse = pd.read_csv(PATH.joinpath('processed', 'kse_shadowfleet.csv'),
                  parse_dates=['earliest_sanction_date'],
                  date_format=lambda x: pd.to_datetime(x, 
                                                       errors='coerce', 
                                                       format='%d-%m-%Y', 
                                                       dayfirst=True))

# create a set of imos
imos = set(kse.imo)

# Some vessels have multiple entries in the dataseet (because they carry multiple products)
# We will drop the duplicates

kse_dedup = kse.drop_duplicates(subset='imo')

print(f'''The KSE dataset contains {len(imos)} unique vessels
of which {kse[kse.earliest_sanction_date.notna()].imo.nunique()} are sanctioned.''')

The KSE dataset contains 637 unique vessels
of which 71 are sanctioned.


In [4]:
# Let's dive into the sanctions: when were they sanctioned?

sanctions = kse_dedup.groupby('earliest_sanction_date').imo.count().reset_index()

sanctions['earliest_sanction_date'] = pd.to_datetime(sanctions['earliest_sanction_date'], 
                                                     errors='coerce', 
                                                     format='%d-%m-%Y', dayfirst=True)

sanctions.earliest_sanction_date = pd.to_datetime(sanctions.earliest_sanction_date)
sanctions.sort_values('earliest_sanction_date', inplace=True)
sanctions.set_index('earliest_sanction_date', inplace=True, drop=True)

# Sanction dates

px.bar(sanctions,
       title='Sanction Dates',
       labels={'earliest_sanction_date':'Sanction Date'})

In [5]:
# What is the Age of the shadowfleet?

px.histogram(kse_dedup, x='buildyear',
             title='Age of the Shadowfleet',
             labels={'buildyear':'Build Year'},
             )

In [6]:
# What is the flag distribution? (N.B. the current flag is the flag as of 6th August 2024)

px.bar(kse_dedup.groupby('current_flag_06082024').imo.count()).\
       update_xaxes(categoryorder='total descending')

In [7]:
# When dit the vessels enter the shadowfleet? (use our uninsured data)
# The first date should be discarded as it is the date of the first entry in the dataset, but the start of measurements

uninsured = pd.read_csv(PATH.joinpath('processed', 'uninsured.csv'),
                        parse_dates=['start_date', 'end_date', 'earliest_sanction_date'],
                        date_format='%Y-%m-%d')

px.bar(uninsured.groupby('start_date').imo.count(),
       title='Entry Dates of the Shadowfleet',
       labels={'start_date':'Entry Date'})

In [8]:
# And what are the end dates? The last date (July 2024) should be discarded because it is not a real end date, but the end of measurements.

px.bar(uninsured.groupby('end_date').imo.count(),
       title='Exit Dates of the Shadowfleet',
       labels={'start_date':'Exit Date'})

#### Company histories and details

The equasis data consists of lists of companies owning or managing a vessel through time. That data is based on the IMO registry and could be very useful in finding relevant changes in vessel ownership. 

In [297]:
companies = pd.read_csv(PATH.joinpath('processed', 'owners_companies.csv'))
companies[['start_date', 'end_date']] = companies[['start_date', 'end_date']].apply(pd.to_datetime)
len(companies)

7963

In [298]:
companies.head()

,company,role,start_date,imo,end_date
0,UNKNOWN,ISM Manager,2000-10-27,8727941,2000-10-27
1,BURREN ENERGY SHIPPING,ISM Manager,2000-10-27,8727941,2005-10-17
2,PRIME SHIPPING LLC- RUS,ISM Manager,2005-10-17,8727941,2009-01-04
3,BURREN ENERGY SHIPPING,ISM Manager,2009-01-04,8727941,2010-06-08
4,UNKNOWN,ISM Manager,2010-06-08,8727941,2010-08-06


In [178]:
# Are there any vessels in the shadowfleet that we don't have company information for?

print(f'There is {len(set(kse.imo).difference(set(companies.imo)))} imos not in the other dataset')

There is 0 imos not in the other dataset


In [179]:
# How many UNKNOWN companies are in this dataset?

companies[companies.company == 'UNKNOWN'].company.count()

514

This seems to be relevant. Let's see these missing values over time. If they are old, than its just an administrative issue. If the missing companies are more recent, then it would be interesting to know why these companies aren't known. The unknown companies are all ISM Managers.  

In [180]:
unknown = companies[companies.company == 'UNKNOWN'].copy()
unknown.start_date = pd.to_datetime(unknown.start_date, errors='coerce', format='%Y-%m-%d', dayfirst=True).dt.year
px.histogram(unknown, x='start_date')

In [181]:
# What kind of roles are unknown?

unknown.role.value_counts()

role
ISM Manager    514
Name: count, dtype: int64

In [182]:
# What companies had many ownership changes after the start of the Ukraine war?

companies.query('role == "Registered owner" & start_date > "01-01-2022"')\
         .groupby('imo')\
         .agg(ownership_changes=('company', 'nunique'))\
         .sort_values('ownership_changes', ascending=False)[0:20]

,ownership_changes
imo,
9302126,4
9296391,3
9334557,3
9257137,3
9292199,3
9326885,3
9388742,3
9318034,3
9423736,3


In [183]:
# Let's look at the vessel with the most ownership changes after the Ukraine war

pd.merge(companies.query('imo == 9302126 & role == "Registered owner"'),
         kse[['imo', 'vessel_name']],
         on='imo',
         how='left')

,company,role,start_date,imo,end_date,vessel_name
0,TEVERE SHIPPING LLC,Registered owner,2005-07-15,9302126,2008-10-14,Rainbow
1,TORM A/S,Registered owner,2008-10-14,9302126,2020-06-29,Rainbow
2,TORM A/S,Registered owner,2020-06-29,9302126,2023-02-01,Rainbow
3,AIDAN MARITIME 88 DAC,Registered owner,2023-02-01,9302126,2023-02-01,Rainbow
4,OLYMPIC GLORY SHIPPING SA,Registered owner,2023-02-01,9302126,2024-01-06,Rainbow
5,JOLLY HOLDING CORP,Registered owner,2024-01-06,9302126,2024-03-01,Rainbow
6,OCEANICORBIT VENTURES LTD,Registered owner,2024-03-01,9302126,NaT,Rainbow


In [184]:
# When did ownership change?

px.bar(companies.groupby([pd.Grouper(key='end_date', freq='ME'), 'role'])\
                .agg(count=('role', 'count'))\
                .reset_index()\
                .set_index('end_date', drop=True).query('index > "01-01-2014"'),
       color='role',
       title='Ownership Changes Over Time',
       labels={'end_date':'End Date',
               'value': 'Number of Ownership Changes'})

In [185]:
com_details = pd.read_csv(PATH.joinpath('processed', 'owners_companies_details.csv'))
com_details[['last_update', 'since']] = com_details[['last_update', 'since']].apply(pd.to_datetime)
com_details[['imo', 'company_imo', 'year_of_build']] = com_details[['imo', 'company_imo', 'year_of_build']].astype(int)
com_details.head()

,imo,ship,gross_tonnage,ship_type,year_of_build,current_flag,current_class,detention_in_last_3_years_for_this_company,detention_in_last_3_years_for_all_company,company_imo,name,address,last_update,since,role
0,9327449,FAIR WONDER,27916,Chemical/ Oil Products Tanker,2006,Liberia,RINA LRS,NaN,NaN,6477523,ONN DENIZCILIK VE DANISMANLIK,"IC Kapi 76, Palladium Tower 2, Kardelen Sokak\...",2024-10-01,2024-02-29,Ship manager/ Commercial manager
1,9396672,FAIR STAR,42010,Crude Oil Tanker,2008,Liberia,IRS,NaN,NaN,6477523,ONN DENIZCILIK VE DANISMANLIK,"IC Kapi 76, Palladium Tower 2, Kardelen Sokak\...",2024-10-01,2024-02-29,Ship manager/ Commercial manager
2,9396775,ELEGANT GRACE,30056,Chemical/ Oil Products Tanker,2009,Marshall Islands,RINA,NaN,NaN,6477523,ONN DENIZCILIK VE DANISMANLIK,"IC Kapi 76, Palladium Tower 2, Kardelen Sokak\...",2024-10-01,2024-02-29,Ship manager/ Commercial manager
3,9408530,SOUTHPORT,62856,Crude Oil Tanker,2008,Marshall Islands,RINA ABS,NaN,NaN,6477523,ONN DENIZCILIK VE DANISMANLIK,"IC Kapi 76, Palladium Tower 2, Kardelen Sokak\...",2024-10-01,2024-02-29,Ship manager/ Commercial manager
4,9408554,PACIFIC SKY,62856,Crude Oil Tanker,2009,Malta,RINA,NaN,NaN,6477523,ONN DENIZCILIK VE DANISMANLIK,"IC Kapi 76, Palladium Tower 2, Kardelen Sokak\...",2024-10-01,2024-02-29,Ship manager/ Commercial manager


### Events

Analyse port visits, loitering, AIS gaps and activity in known ship-to-ship transfer zones

In [268]:
ports = pd.read_parquet(PATH.joinpath('processed', 'ports.parquet'))
loitering = pd.read_parquet(PATH.joinpath('processed', 'loitering.parquet'))
ais = pd.read_parquet(PATH.joinpath('processed', 'ais.parquet'))
sts = pd.read_parquet(PATH.joinpath('processed', 'sts_tracks.parquet'))

#### Port visits

In [241]:
# Clean ports

ports[['start', 'end']] = ports[['start', 'end']].apply(lambda x: pd.to_datetime(x).dt.date)

# See port visits over time

ports[ports.port_visit_startAnchorage_id.str.contains('tur-')].port_visit_startAnchorage_name.value_counts().nlargest(10)

port_visit_startAnchorage_name
ISTANBUL        4769
CANAKKALE       3031
YALOVA TOWN     1620
RUMELIFENERI    1061
AMBARLI          848
ALIAGA           783
DARDANELLES      652
ICDAS            650
MERSIN           625
TUZLA            547
Name: count, dtype: int64

In [252]:
ports['port_name'] = ports.groupby(['port_visit_startAnchorage_id'])['port_visit_startAnchorage_name'].ffill()

52008

In [255]:
ports[ports.start > pd.to_datetime('2022-01-01').date()].to_csv(PATH.joinpath('processed', 'port_visits_2022.csv'), index=False)

In [ ]:
# Enter 3 iso country code and port name

COUNTRY = 'tur'
PORT = 'AMBARLI'

px.bar(ports[(ports.port_visit_startAnchorage_id.str.contains(f'{COUNTRY}-')) \
             & (ports.port_visit_startAnchorage_name==PORT)].groupby(pd.Grouper(key='start', freq='ME')).agg(visits=('ssvid', 'count')),
             title=f'Port visits from shadowfleet vessels to {PORT} by month',
             labels={'visits': 'Number of visits',
                     'start': 'Time'},)

#### Loitering

In [272]:
loitering[['start', 'end']] = loitering[['start', 'end']].apply(pd.to_datetime)
loitering = loitering[loitering.start >= '2022-01-01'].copy()
loitering[['start', 'end']] = loitering[['start', 'end']].apply(lambda x: x.dt.date)
len(loitering)

39084

In [273]:
loitering.to_csv(PATH.joinpath('processed', 'loitering_2022.csv'), index=False)

In [274]:
loitering.head()

,start,end,boundingBox,lat,lon,id,name,ssvid,flag,type,loitering_totalTimeHours,loitering_totalDistanceKm,loitering_averageSpeedKnots,loitering_averageDistanceFromShoreKm,imo
10,2024-03-04,2024-03-04,"[33.109425, 42.88973333333333, 33.099005, 42.8...",42.8896,33.1042,bfc29a460-00d9-7f81-0413-f3e009df4050,KAPITAN PSHENITSIN,273344600,RUS,other,0.782500,0.984241,0.679167,97.000000,8727941
11,2024-05-20,2024-05-20,"[35.18595, 43.470715, 35.17800833333333, 43.47...",43.4712,35.1820,bfc29a460-00d9-7f81-0413-f3e009df4050,KAPITAN PSHENITSIN,273344600,RUS,other,0.971944,0.992296,0.551263,136.022079,8727941
12,2024-07-05,2024-07-05,"[36.681198333333334, 45.81865166666667, 36.682...",45.8184,36.6817,bfc29a460-00d9-7f81-0413-f3e009df4050,KAPITAN PSHENITSIN,273344600,RUS,other,0.434167,0.100534,0.125031,42.000000,8727941
13,2024-08-20,2024-08-20,"[36.682048333333334, 45.817965, 36.686875, 45....",45.8184,36.6845,bfc29a460-00d9-7f81-0413-f3e009df4050,KAPITAN PSHENITSIN,273344600,RUS,other,4.979444,6.403149,0.694339,41.957897,8727941
17,2022-03-28,2022-03-28,"[36.58213333333333, 43.301098333333336, 36.764...",43.2911,36.6734,e4c0e77b0-06af-933b-10ee-6c9b0613c1aa,KAPITAN PERMYAKOV,273343600,RUS,other,14.390967,15.458626,0.580016,169.858462,8727953


#### AIS

In [ ]:

ais['geometry'] = ais.apply(lambda x: LineString([(x['gap_offposition_lon'], x['gap_offposition_lat']) , (x['gap_onposition_lon'], x['gap_onposition_lat'])]), axis = 1)
ais = gpd.GeoDataFrame(ais, geometry='geometry', crs='EPSG:4326')

In [ ]:
ais[['geometry', 'name']].explore()

#### Ship-to-ship transfers

In [152]:
# Clean STS

sts.drop('seg_id', axis=1, inplace=True)
sts.timestamp = pd.to_datetime(sts.timestamp)
sts.query('timestamp > "2022-01-01"', inplace=True)
sts.sort_values('timestamp', inplace=True)



In [153]:
sts['week'] = sts.timestamp.dt.to_period('W').apply(lambda x: x.start_time)

In [154]:
sts.groupby(['name', 'week']).agg(count=('name', 'count')).reset_index().sort_values('count', ascending=False).name.value_counts()[140:150]

name
MOREA                 14
AIDA                  14
SAINTGEORGE           14
FAIR STAR             14
NARCISSUS             14
RIDGEBURYNICHOLASA    14
ELISE                 14
YANGTZE               13
PRINCE1               13
SCFSAYAN              13
Name: count, dtype: int64

In [155]:
sts.columns

Index(['lon', 'lat', 'course', 'timestamp', 'speed', 'name', 'week'], dtype='object')

In [156]:
sts = gpd.GeoDataFrame(sts, geometry=gpd.points_from_xy(x=sts.lon, y=sts.lat), crs='EPSG:4326')
sts.week = sts.week.astype(str)



In [157]:
sts_locations = gpd.read_file(PATH.joinpath('geo', 'sts_locations.geojson'))
sts_locations.set_geometry('geometry', crs='EPSG:4326', inplace=True)
sts = gpd.sjoin(sts, sts_locations, how='left', predicate='within')

In [159]:
sts.drop('index_right', axis=1, inplace=True)
sts.rename(columns={'Name':'sts_location'}, inplace=True)

In [161]:
sts.groupby('name').agg(count=('sts_location', 'nunique')).sort_values('count', ascending=False).head(10)

,count
name,
HAMPSTEAD,5
SCFPECHORA,5
TREX,5
AIDA,5
RICCA,5
SEASCOUT,4
SCFBALTICA,4
ELISE,4
HS LEGEND,4


In [162]:
sts.query('name=="AIDA"')[['geometry', 'week']].explore(column='week')